In [57]:
import os
import pandas as pd

# 改變Date、File

In [16]:
import pandas as pd
from datetime import datetime

In [17]:
data = pd.read_excel('加權指數.xlsx')

data['Date'] = pd.to_datetime(data['Date'], format='%Y/%m/%d')

for i in range(len(data.index)):
    name = data['Date'].iloc[i]
    data.loc[i, 'File'] = f"o{name.strftime('%Y%m%d')}.csv"

data['Date'] = data['Date'].dt.date
data.to_excel('加權指數.xlsx', index = False)

In [ ]:
data

# 算到期日

In [28]:
import pandas as pd
from pandas.tseries.offsets import MonthEnd, Week

data = pd.read_excel('加權指數.xlsx')
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

def third_wednesday(date):
    last_day = (date + MonthEnd(0)).replace(day=1)
    first_wednesday = last_day + Week(weekday=2)
    if first_wednesday.day <= 7:
        return first_wednesday + Week(2)
    else:
        return first_wednesday + Week(1)

def update_contract_and_maturity(row):
    if row['Maturity'] < 0:
        next_month = row['Date'] + pd.DateOffset(months=1)
        maturity_date = third_wednesday(next_month)
        row['Contract'] = maturity_date.strftime('%Y-%m')
        row['Maturity'] = (maturity_date - row['Date']).days
    return row

data['MaturityDate'] = data['Date'].apply(third_wednesday)

data['Maturity'] = (data['MaturityDate'] - data['Date']).dt.days

data['Contract'] = data['MaturityDate'].dt.strftime('%Y-%m')

data = data.drop(columns=['MaturityDate'])

data = data.apply(update_contract_and_maturity, axis=1)

data['Date'] = data['Date'].dt.date

data.to_excel('加權指數.xlsx', index = False)

# 解壓縮

In [ ]:
folder_path = '/Users/xinc./Documents/GitHub/desktop-tutorial/ndhu/midterm/逐筆資料/Option_2021'

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    try:
        data = pd.read_csv(file_path, encoding = 'big5')
        data.to_csv(file_path, index = False)
        print(filename)

    except UnicodeDecodeError as e:
        print(f"編碼錯誤（Big5 無法解碼）：{filename}，錯誤訊息：{e}")

    except Exception as e:
        print(f"處理檔案 {filename} 時發生錯誤：{e}")

# 整理逐日資料

In [85]:
folder_path = '/Users/xinc./Documents/GitHub/desktop-tutorial/ndhu/midterm/逐筆資料/Option_2021'

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)

    data = pd.read_csv(file_path, dtype = str, low_memory = False)
    data.columns = data.columns.str.strip()
    filterdata = data[data['商品代號'].str.strip() == 'TXO']
    filterdata = filterdata.reset_index(drop = True)
    filterdata = filterdata.dropna()
    filterdata.to_csv(file_path)